In [38]:
import json
import pandas as pd

In [39]:
tweets = []
for line in open('News_Category_Dataset_v2.json', 'r'):
    tweets.append(json.loads(line))

In [40]:
model_dataset=pd.DataFrame()
model_dataset['News']='Not_Available'
model_dataset['Category']='Not_Available'
news=[]
category=[]
for i in range(len(tweets)):
    a=tweets[i]
    category.append(a['category'])
    news.append(a['headline'])
model_dataset['News']=news
model_dataset['Category']=category


In [41]:
model_dataset

,News,Category
0,There Were 2 Mass Shootings In Texas Last Week...,CRIME
1,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Hugh Grant Marries For The First Time At Age 57,ENTERTAINMENT
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,ENTERTAINMENT
4,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT
...,...,...
200848,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH
200849,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS
200850,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS
200851,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS


In [49]:
print(model_dataset.News.shape)

(200853,)


In [52]:
!pip install sklearn


In [53]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(model_dataset.News)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [54]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))



In [57]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, model_dataset.Category, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [59]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = list(set(category))

docs_new = "Messi joins other football team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

# print(category_list[predicted[0]])

In [62]:
predicted = loaded_model.predict(X_test)
result_bayes = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})

In [63]:
result_bayes

,true_labels,predicted_labels
56274,CRIME,POLITICS
9929,POLITICS,POLITICS
163271,STYLE & BEAUTY,STYLE & BEAUTY
190013,DIVORCE,WELLNESS
13564,POLITICS,POLITICS
...,...,...
188812,PARENTING,POLITICS
23556,STYLE,ENTERTAINMENT
309,ENTERTAINMENT,ENTERTAINMENT
175940,HOME & LIVING,WELLNESS


In [66]:


from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)



[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  3 ...  0  0  0]
 ...
 [ 0  0  0 ... 29  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
